# Data Collection: Tables from Academic Papers (PDF Form)

## Explanation of Notebook

This notebook was a project for my internship with Earth Economics. I was asked to gather data about wild rice harvesting throughout the Native American tribes of Minnesota. Everything I found was in the form a report (PDF), either from the Minnesota Department of Natural Resources (MNDNR) or from one of the tribal agencies. The PDF below, had enough data that it made more sense to transfer it to text in Python and use regular expressions to separate it accordingly.

*The table I am turning into a csv starts on page 57 (numbered 54 on the page), which since the page iterator starts at 0, will be started at 56.*

In [1]:
#######========> Display pdf file as image in the notebook
from IPython.display import IFrame
IFrame("MNDNR-Natural_Wild_Rice_in_Minnesota.pdf", width=700, height=700)

## Explanation of Process

So the basic structure in order for data to be read as a csv is that each row must be separated by a newline character, each column in each row separated by a comma. In addition, there needs to be a text delimiter so that a chunk of text can be taken together no matter what characters are in the string. 

It is visible in the PDF above that it *looks* like it is already in proper csv format, but a PDF has the text printed on it, so it doesn't have any systematic structure to it, when it is converted to text.

The first step in this notebook will be to use the PyPDF2 library to read the desired pages of the PDF into text format. After that, it will be a matter of understanding the text output in this particular example and designing substitutions using python's reguar expressions library (re) to rearrange the unstructured text into a csv format.

Additionally, throughout this notebook, I will explain the idea behind the different methods I am using in the regular expressions library to make the approprite substitutions.

## Diving Into the Code

First thing to do is import the necessary libraries for the project, shown below. Additionally, after trying this for a while, I realized I wouldn't be able complete the rearrangement in a reasonable time unless I defined the set of County Names (the first column in the table in the PDF). While this is not necesssarily applicable to all text conversion situations, the distinct set (assigned to the list counties) is not too large, so it made sense for this application.

In [2]:
#######========> Import libraries and set parameters needed for this notebook

import PyPDF2 as pdf #Library to read PDFs into text
import pandas as pd #Library for data manipulation
import re #Library for regular expressions, text manipulation

#######========> End library importing

#Show plots within the notebook
%matplotlib inline
#Set the max column width higher to visualize everything
pd.options.display.max_colwidth = 5000

#######========> Initialize variable to hold text and read in pdf file

#Define the file path where this group of files is held
path = '/Users/andrewboomer/Desktop/Earth Economics/AngelaWork/Rice Reports/Acreage_by_Waters/'

#Read in the pdf file using the PyPDF2 library
file = pdf.PdfFileReader(path + 'MNDNR-Natural_Wild_Rice_in_Minnesota.pdf')

#Declare a list of the counties, written in by hand, to aid in separation within tedt
counties = ['Aitkin','Anoka','Becker','Beltrami','Blue Earth','Brown',
            'Carlton','Cass','Chippewa','Chisago','Clay','Clearwater',
            'Cook','Crow Wing','Dakota','Douglas','Faribault','Fillmore',
            'Freeborn','Goodhue','Hennepin','Houston','Hubbard','Isanti',
            'Itasca','Kanabec','Kandiyohi','Koochiching','Lake',
            'Lake of the Woods','Le Sueur','Mahnomen','McLeod','Mille Lacs',
            'Morrison','Nicollet','Otter Tail','Pine','Pope','Polk','Ramsey',
            'Redwood','Rice','Roseau','Scott','Sherburne','Sibley','St. Louis',
            'Stearns','Steele','Todd','Wabasha','Wadena','Wright']

#######========> End initialization

To start off, I will just pring the extracted text from the first page of tables to see the format after extraction. It is easy to see in the printed text below the lack of structure and the needed substitutions and adjustments that will have to be made. The goal of this notebook is to adjust the text below into the format of the tables in the above PDF in as few adjustments as possible. So the more universal an adjustment is within the extracted text, the better.

In [3]:
#Read page 57 (56 because of 0th page) into text
first = file.getPage(56).extractText()
print(first)

54County name Location Name (i.e. Lake or River) MN L
ake ID 
Location 
size (acres) 
Estimated 
wild rice 
coverage 
(acres)
Aitkin Aitkin 01004000 850 298
Aitkin Anderson 01003100 97 30
Aitkin Bear 
01006400 127 1
Aitkin Big 
Sandy 01006200 9,380 94
Aitkin Birch 
01020600 449 5
Aitkin Blind 
01018800 323 39
Aitkin Brown 01007800 97 34
Aitkin Camp 01009800 127 30
Aitkin Clear 
01010600 123 20
Aitkin Cornish 
Pool 01042700 600 30
Aitkin Davis 01007101 76 30
Aitkin Deer 
01008600 47 3
Aitkin Elm 
Island 01012300 656 30
Aitkin Farm 
Island 01015900 2,025 20
Aitkin Fleming 01010500 326 1
Aitkin Flowage 01006100 720 432
Aitkin Gun 
01009900 735 60
Aitkin Hammal 01016100 376 1
Aitkin Hay 
01005900 133 1
Aitkin Hickory 01017900 183 10
Aitkin Jenkins 01010000 127 1
Aitkin Jewett State WMA - Impoundment 01038300 180 
30
Aitkin Johnson 01013100 27 6
Aitkin Killroy 01023800 23 4
Aitkin Kimberly State WMA - Lower Pool 01043300 300
 30
Aitkin Kimberly State WMA - Upper Pool 01041100 900
 76
Aitkin

So one thing to note, before converting the full pdf into text, is that each page has the headers for the table. Since these are only needed once, I will get rid of them on every page and hand type in the headers. For this, I will read in the text of the first table page as is, and see how many characters the headers take up. For page 57, we want to start at the first Aitkin (County Name). So, one way is just count the number of characters that the headers take up, but I just did a few iterations of reasonable guesses and arrived at the correct number, 126

In [4]:
#Trial and error figure out the right number of characters
#The correct number of characters ends up being 126
print(first[126:])

Aitkin Aitkin 01004000 850 298
Aitkin Anderson 01003100 97 30
Aitkin Bear 
01006400 127 1
Aitkin Big 
Sandy 01006200 9,380 94
Aitkin Birch 
01020600 449 5
Aitkin Blind 
01018800 323 39
Aitkin Brown 01007800 97 34
Aitkin Camp 01009800 127 30
Aitkin Clear 
01010600 123 20
Aitkin Cornish 
Pool 01042700 600 30
Aitkin Davis 01007101 76 30
Aitkin Deer 
01008600 47 3
Aitkin Elm 
Island 01012300 656 30
Aitkin Farm 
Island 01015900 2,025 20
Aitkin Fleming 01010500 326 1
Aitkin Flowage 01006100 720 432
Aitkin Gun 
01009900 735 60
Aitkin Hammal 01016100 376 1
Aitkin Hay 
01005900 133 1
Aitkin Hickory 01017900 183 10
Aitkin Jenkins 01010000 127 1
Aitkin Jewett State WMA - Impoundment 01038300 180 
30
Aitkin Johnson 01013100 27 6
Aitkin Killroy 01023800 23 4
Aitkin Kimberly State WMA - Lower Pool 01043300 300
 30
Aitkin Kimberly State WMA - Upper Pool 01041100 900
 76
Aitkin Krilwitz 01IMP002 30 6
Aitkin Lily 
01008800 50 2
Aitkin Little Hill River State WMA - Pool 1 0104330
0 135 18
Aitkin Little 

After figuring out the correct number of characters in the header is 126, I will just chop off this amount from each page of extracted text. I combine all the pages of text by initializing and empty string, and then concatenating each page of coverted text onto the end of that.

In [5]:
#######========> Extract the text from each relevant page
text = ''

for i in range(56, 87):
    first = file.getPage(i)
    text += str(first.extractText()[126:])
    
#######========> End text extraction

print(text)

Aitkin Aitkin 01004000 850 298
Aitkin Anderson 01003100 97 30
Aitkin Bear 
01006400 127 1
Aitkin Big 
Sandy 01006200 9,380 94
Aitkin Birch 
01020600 449 5
Aitkin Blind 
01018800 323 39
Aitkin Brown 01007800 97 34
Aitkin Camp 01009800 127 30
Aitkin Clear 
01010600 123 20
Aitkin Cornish 
Pool 01042700 600 30
Aitkin Davis 01007101 76 30
Aitkin Deer 
01008600 47 3
Aitkin Elm 
Island 01012300 656 30
Aitkin Farm 
Island 01015900 2,025 20
Aitkin Fleming 01010500 326 1
Aitkin Flowage 01006100 720 432
Aitkin Gun 
01009900 735 60
Aitkin Hammal 01016100 376 1
Aitkin Hay 
01005900 133 1
Aitkin Hickory 01017900 183 10
Aitkin Jenkins 01010000 127 1
Aitkin Jewett State WMA - Impoundment 01038300 180 
30
Aitkin Johnson 01013100 27 6
Aitkin Killroy 01023800 23 4
Aitkin Kimberly State WMA - Lower Pool 01043300 300
 30
Aitkin Kimberly State WMA - Upper Pool 01041100 900
 76
Aitkin Krilwitz 01IMP002 30 6
Aitkin Lily 
01008800 50 2
Aitkin Little Hill River State WMA - Pool 1 0104330
0 135 18
Aitkin Little 

So, comparing the printed version of the extracted text to the table in the pdf, we can already see all the mistakes. There are new line characters where there shouldn't be, double spaces, etc... Next task is to define some functions to clean this up a bit. The goal is to come up with systematic fixes, where the goal is to get it to look exactly like the table in the pdf.

## Starting the substitutions...

First off, to explain the process, it is helpful to think of two different classes of substitutions. One class is searhing for specific characters or sequences of characters and replacing those with another character (or empty space). Another class is looking for general patterns of character types in the text and inserting or deleting characters between those.

The first class, direct character substitution, is what I will start off with, and is in the cell below. Doing this in python with the 're' libray, there is a function called 'sub' as shown below. If you want to substitute multiple characters at once, you can enumerate them in a list, as done in the first line below, where I replace the five characters within the list with a blank (deletion). 

Another point to note in that first line is the '+' sign after the list. In the python re library, you can specify 0 or more occurences (* character), 1 or more occurences (+ character), or 0/1 times (? character). One more point is that I needed to use an extra escape character '\' for the newline character '\n\ to be recognized.

In [51]:
#######========> Do first round of simple character and duplicate deletion

# Get rid of all special characters in the text
new_text = re.sub('[-,().]+', '', text)

#Replace double newline characters, with or without spaces, with a single new line character
new_text = re.sub(' *\\n+ *\\n+', '\\n', new_text)

#Replace one or more spaces with a single space
new_text = re.sub(' +', ' ', new_text)

#######========> End simple character and duplicate replacements

#Visualize the new text and perform checks
print(new_text)

Aitkin Aitkin 01004000 850 298
Aitkin Anderson 01003100 97 30
Aitkin Bear 
01006400 127 1
Aitkin Big 
Sandy 01006200 9380 94
Aitkin Birch 
01020600 449 5
Aitkin Blind 
01018800 323 39
Aitkin Brown 01007800 97 34
Aitkin Camp 01009800 127 30
Aitkin Clear 
01010600 123 20
Aitkin Cornish 
Pool 01042700 600 30
Aitkin Davis 01007101 76 30
Aitkin Deer 
01008600 47 3
Aitkin Elm 
Island 01012300 656 30
Aitkin Farm 
Island 01015900 2025 20
Aitkin Fleming 01010500 326 1
Aitkin Flowage 01006100 720 432
Aitkin Gun 
01009900 735 60
Aitkin Hammal 01016100 376 1
Aitkin Hay 
01005900 133 1
Aitkin Hickory 01017900 183 10
Aitkin Jenkins 01010000 127 1
Aitkin Jewett State WMA Impoundment 01038300 180 
30
Aitkin Johnson 01013100 27 6
Aitkin Killroy 01023800 23 4
Aitkin Kimberly State WMA Lower Pool 01043300 300
 30
Aitkin Kimberly State WMA Upper Pool 01041100 900
 76
Aitkin Krilwitz 01IMP002 30 6
Aitkin Lily 
01008800 50 2
Aitkin Little Hill River State WMA Pool 1 0104330
0 135 18
Aitkin Little 
McKinney 

The next very important method that can be used in the python re library that I learned while doing this project is the idea of a lookaround/lookbehind/lookahead (used below). The idea of these is that instead of being constrained to replacing the character you are searching for in the text, you can replace the character that comes before, after, or in between your search criteria (lookafter, lookbehind, and lookaround respectively).

A lookbehind has the syntax: '(?<=chars)' where 'chars' is the characters or criteria you are searching for.
A lookahead has the syntax: '(?=chars)' where 'chars' is the characters or criteria you are searching for.
A lookaround has the syntax: '(?<=chars1)(?=chars2)' where 'chars1/2' are the characters or criteria you are searching for.

If there is no character after, before, or in between the lookbehind, lookahead, or lookaround, then the character you select for replacement will replace a blank. You can define a character found by these methods by after it after, before or in between the search criteria. 

For example, if I wanted to replace a space in between all instances of two capital 'A's with a colon from a string called 'text', I could do:

re.sub('(?<=A) (?=A)', ':', text)

One last point: in the re library, you can define all the characters/ranges of characters you want to search for in a list, but you can also use the predefined classes of characters in re. The ones I use in this notebook are:

\d = digits [0-9]

\D = non digits

\w = alphanumeric characters [a-zA-Z0-9_]

\W = non alphanumeric characters

In [52]:
#######========> Use lookarounds (lookbehind + lookahead) to perform replacements

#Do a lookaround to delete newline characters between two digits
new_text = re.sub('(?<=\d)\\n+(?=\d)', '', new_text)

#Do a lookaround to replace spaces between digits with a comma
new_text = re.sub('(?<=\d) (?=\d)', ',', new_text)

#Do a lookaround to replace newline characters between letters and digits with a comma
new_text = re.sub('(?<=\w) \\n+(?=\d)|(?<=\w)\\n +(?=\d)', ',', new_text)

#Do a lookaround to replace space then newline characters with a comma
new_text = re.sub('(?<=\w) \\n(?=\w)', ' ', new_text)

#######========> End first round of lookaround for certain replacements

#Visualize the new text and perform checks
print(new_text)

Aitkin Aitkin 01004000,850,298
Aitkin Anderson 01003100,97,30
Aitkin Bear,01006400,127,1
Aitkin Big Sandy 01006200,9380,94
Aitkin Birch,01020600,449,5
Aitkin Blind,01018800,323,39
Aitkin Brown 01007800,97,34
Aitkin Camp 01009800,127,30
Aitkin Clear,01010600,123,20
Aitkin Cornish Pool 01042700,600,30
Aitkin Davis 01007101,76,30
Aitkin Deer,01008600,47,3
Aitkin Elm Island 01012300,656,30
Aitkin Farm Island 01015900,2025,20
Aitkin Fleming 01010500,326,1
Aitkin Flowage 01006100,720,432
Aitkin Gun,01009900,735,60
Aitkin Hammal 01016100,376,1
Aitkin Hay,01005900,133,1
Aitkin Hickory 01017900,183,10
Aitkin Jenkins 01010000,127,1
Aitkin Jewett State WMA Impoundment 01038300,180,30
Aitkin Johnson 01013100,27,6
Aitkin Killroy 01023800,23,4
Aitkin Kimberly State WMA Lower Pool 01043300,300,30
Aitkin Kimberly State WMA Upper Pool 01041100,900,76
Aitkin Krilwitz 01IMP002,30,6
Aitkin Lily,01008800,50,2
Aitkin Little Hill River State WMA Pool 1,01043300,135,18
Aitkin Little McKinney 01019700,26,6
Ait

In [53]:
#######========> Next, use the hand keyed list of counties to add a comma after each

#Loop over the counties in the county list
for county in counties:
    
    #Replace space between last line and new line with newline character
    new_text = re.sub('(?<=\w\d|\d\d) (?=' + county + ')','\\n',new_text)
    
    #Replace the space after the county name with a comma
    new_text = re.sub('(?<=\\n' + county + ') +(?=\w)',',',new_text)
    
    #Insert space between county and location name if none exists
    new_text = re.sub('(?<=\\n' + county + ')(?=\w+ )',',',new_text)

#######========> End looping over counties to add commas afterwards

#Visualize the new text and perform checks
print(new_text)

Aitkin Aitkin 01004000,850,298
Aitkin,Anderson 01003100,97,30
Aitkin,Bear,01006400,127,1
Aitkin,Big Sandy 01006200,9380,94
Aitkin,Birch,01020600,449,5
Aitkin,Blind,01018800,323,39
Aitkin,Brown 01007800,97,34
Aitkin,Camp 01009800,127,30
Aitkin,Clear,01010600,123,20
Aitkin,Cornish Pool 01042700,600,30
Aitkin,Davis 01007101,76,30
Aitkin,Deer,01008600,47,3
Aitkin,Elm Island 01012300,656,30
Aitkin,Farm Island 01015900,2025,20
Aitkin,Fleming 01010500,326,1
Aitkin,Flowage 01006100,720,432
Aitkin,Gun,01009900,735,60
Aitkin,Hammal 01016100,376,1
Aitkin,Hay,01005900,133,1
Aitkin,Hickory 01017900,183,10
Aitkin,Jenkins 01010000,127,1
Aitkin,Jewett State WMA Impoundment 01038300,180,30
Aitkin,Johnson 01013100,27,6
Aitkin,Killroy 01023800,23,4
Aitkin,Kimberly State WMA Lower Pool 01043300,300,30
Aitkin,Kimberly State WMA Upper Pool 01041100,900,76
Aitkin,Krilwitz 01IMP002,30,6
Aitkin,Lily,01008800,50,2
Aitkin,Little Hill River State WMA Pool 1,01043300,135,18
Aitkin,Little McKinney 01019700,26,6
Ait

In [54]:
#######========> Use lookarounds to fix a few one-off situations

#Replace the string Pool follow by an ID number with a comma
new_text = re.sub('(?<=Pool)(?=\w+)',',',new_text)

#Insert comma after first Aitkin which wasn't recognized while looping over the counties
new_text = re.sub('(?<=Aitkin) (?=Aitkin)',',',new_text)

#Replace the space between location names and Lake IDs with a comma
new_text = re.sub('(?<=\w|\d) (?=\d\d\w+\d|\w\d\d+)',',',new_text)

#Replace blanks between location names and Lake IDs
new_text = re.sub('(?<=[a-z][a-z])(?=\d+\w*|\w\d+)',',',new_text)

#Replace blanks between location names and Lake IDs
new_text = re.sub('(?<=[a-z]),(?=[a-z])','',new_text)

#######========> End one off replacements

#Visualize the new text and perform checks
print(new_text)

Aitkin,Aitkin,01004000,850,298
Aitkin,Anderson,01003100,97,30
Aitkin,Bear,01006400,127,1
Aitkin,Big Sandy,01006200,9380,94
Aitkin,Birch,01020600,449,5
Aitkin,Blind,01018800,323,39
Aitkin,Brown,01007800,97,34
Aitkin,Camp,01009800,127,30
Aitkin,Clear,01010600,123,20
Aitkin,Cornish Pool,01042700,600,30
Aitkin,Davis,01007101,76,30
Aitkin,Deer,01008600,47,3
Aitkin,Elm Island,01012300,656,30
Aitkin,Farm Island,01015900,2025,20
Aitkin,Fleming,01010500,326,1
Aitkin,Flowage,01006100,720,432
Aitkin,Gun,01009900,735,60
Aitkin,Hammal,01016100,376,1
Aitkin,Hay,01005900,133,1
Aitkin,Hickory,01017900,183,10
Aitkin,Jenkins,01010000,127,1
Aitkin,Jewett State WMA Impoundment,01038300,180,30
Aitkin,Johnson,01013100,27,6
Aitkin,Killroy,01023800,23,4
Aitkin,Kimberly State WMA Lower Pool,01043300,300,30
Aitkin,Kimberly State WMA Upper Pool,01041100,900,76
Aitkin,Krilwitz,01IMP002,30,6
Aitkin,Lily,01008800,50,2
Aitkin,Little Hill River State WMA Pool 1,01043300,135,18
Aitkin,Little McKinney,01019700,26,6
Ait

In [62]:
#######========> Being turning the properly comma separated text into a dataframe to turn it into a csv

#Split the text on the new line character, each list entry corresponding to a row in the dataframe
df = pd.DataFrame(new_text.split('\n'))

#Define the column names for the data frame
columns = ['County Name','Location Name (i.e. Lake or River)',
           'MN Lake ID','Location size (acres)',
           'Estimated wild rice coverage (acres)']

#Split each row of the dataframe on commas, creating a separated list
df.iloc[:,0] = df.iloc[:,0].apply(lambda x: x.split(','))

#######========> End dataframe creation

#######========> Define df_split, to assign a string or float type to the columns

#Define a function to properly assign data types to each column
def df_split(x, i):
    
    #For columns: County Name, Location Name, and Lake ID
    if i <= 2:
        
        #Make the columns a string
        try:
            x = str(x[i])
        except:
            x = str(np.nan)
            
    #For columns: Location Size, and Estimated Coverage
    else:
        
        #Make the columns a float
        try:
            x = float(x[i])
        except:
            x = float(np.nan)
    return x

#######========> End df_split definintion

df.head(10)

,0
0,"[Aitkin, Aitkin, 01004000, 850, 298]"
1,"[Aitkin, Anderson, 01003100, 97, 30]"
2,"[Aitkin, Bear, 01006400, 127, 1]"
3,"[Aitkin, Big Sandy, 01006200, 9380, 94]"
4,"[Aitkin, Birch, 01020600, 449, 5]"
5,"[Aitkin, Blind, 01018800, 323, 39]"
6,"[Aitkin, Brown, 01007800, 97, 34]"
7,"[Aitkin, Camp, 01009800, 127, 30]"
8,"[Aitkin, Clear, 01010600, 123, 20]"
9,"[Aitkin, Cornish Pool, 01042700, 600, 30]"


In [63]:
#######========> Use the df_split function on each entry of the column list to create new column

#Loop over the column name and index number
for i, column in enumerate(columns):
    
    #Assign the new column to the appropriate entry of the split list from the first column
    df[column] = df.iloc[:,0].apply(lambda x: df_split(x, i))
    
#######========> End column creation

#######========> Make final adjustments to the dataframe

#Drop the first column that had the split list
df = df.drop(0, axis = 1)
#df.iloc[1057,4] = df.iloc[1057,2].split(' ')[1]
#df.iloc[1057,2] = df.iloc[1057,2].split(' ')[0]

#Ensure the 5th column is of type float
df[df.columns[4]] = df[df.columns[4]].apply(lambda x: float(x))

#Create a new percent cover column that takes the wild rice coverage divided by total area
df['Percent_Cover'] = (df[df.columns[4]]/df[df.columns[3]])*100

#######========> End final adjustments to the dataframe

#Display the first five rows of the dataframe
df.head(10)

,County Name,Location Name (i.e. Lake or River),MN Lake ID,Location size (acres),Estimated wild rice coverage (acres),Percent_Cover
0,Aitkin,Aitkin,01004000,850.0,298.0,35.058824
1,Aitkin,Anderson,01003100,97.0,30.0,30.927835
2,Aitkin,Bear,01006400,127.0,1.0,0.787402
3,Aitkin,Big Sandy,01006200,9380.0,94.0,1.002132
4,Aitkin,Birch,01020600,449.0,5.0,1.113586
5,Aitkin,Blind,01018800,323.0,39.0,12.074303
6,Aitkin,Brown,01007800,97.0,34.0,35.051546
7,Aitkin,Camp,01009800,127.0,30.0,23.622047
8,Aitkin,Clear,01010600,123.0,20.0,16.260163
9,Aitkin,Cornish Pool,01042700,600.0,30.0,5.000000


In [64]:
#Write the dataframe to a csv file and don't include the index
df.to_csv(path + 'Wild_Rice_Acreage.csv', index = False)